# INTRODUCTION
This notebook contains sales forecasting model proof of concept and experiments for Briton and kay supermarket Oleh Branch
...

# LIBRARY IMPORTATION

In [1]:
#import data manipulation libraries
import os

import pandas as pd
import numpy as np
import pdfplumber

import re

# import data visualization libraries
import seaborn as sns 
import matplotlib as plt 


from pathlib import Path

# EXPLORATORY DATA ANALYSIS

### Importation of raw data

In [ ]:
#raw data path
raw_data_path = Path.joinpath(Path.cwd().parent,"data/raw/details.pdf")
#create an empty list of dataframes
data_frames = []
try: 
    with pdfplumber.open(raw_data_path) as pdf:
            for page_index, page in enumerate(pdf.pages):
                rows=[]
                lines = page.extract_text_lines()
                sales_group = []  # Temporary storage for rows starting with a date
                for j, line in enumerate(lines):
                    if j > 3:  # Skip the first 3 lines
                        date_pattern = r"^\d{2}/\d{2}/\d{4}" # regular expression to detect date
                        line_text = line['text']
                        if re.match(date_pattern, line_text):  # Check if the line starts with a date
                            sales_group = line_text.split(' ')
                            if len(sales_group) >10:
                                sales_group[:5] =  sales_group[:5] 
                                sales_group[5] = " ".join(sales_group[5:-4])
                                sales_group[-4:] = sales_group[-4:]
                        else:
                            # Split the string into 4 parts
                            split_line = line_text.split()
                            if len(split_line) >= 4:
                                row = [
                                    split_line[0],  # First item
                                    " ".join(split_line[1:-2]),  # Middle items
                                    split_line[-2],  # Second-to-last item
                                    split_line[-1],  # Last item
                                ]
                            else:
                                # Handle cases where the line has fewer than 4 items
                                row = split_line + [''] * (4 - len(split_line)) #pad to ensure 4 elements
                            if len(sales_group)<10:
                                sales_group = sales_group + [''] * (10 - len(sales_group)) #pad to ensure 10 elements
                            # Append the new row to the temporary rows
                            combined_row = sales_group + row  # Combine sales_group and row
                            combined_row = combined_row[:14]  # Ensure length is 13
                            rows.append(combined_row)  # Add to the final rows for the page

                # if page_index == 10:
                #      print('maximum page reached')
                #      break
                print(f'\r processed page {page_index} with {len(lines)} text lines', end=" ")  
                page_df = pd.DataFrame(rows, columns=['Date','Receipt_no', 'Receipt_type', 'Total_qnty', 
                                                    'Total_amount', 'Payment_type', 'Time','Time_of_day', 'Cashpoint_number', 'Cashier', 
                                                            'item_no', 'product_name', 'quantity','amount'])
                data_frames.append(page_df)
except FileNotFoundError:
    print('\n file path does not exist')
# except Exception as e :
#     print(f'\n something went wrong: \n {e}')


 processed page 10121 with 43 text lines  

In [30]:
len(data_frames)

10122

In [ ]:
main_df = pd.concat(data_frames, ignore_index= True)
main_df.head()

,Date,Receipt_no,Receipt_type,Total_qnty,Total_amount,Payment_type,Time,Time_of_day,Cashpoint_number,Cashier,item_no,product_name,quantity,amount
0,12/01/2022,437170,Sales,1,600.00,Cash,10:10:31,pm,14,DIRECTOR,8893,KOLAQ ALAGBO BITTERS 200ML,1,600.00
1,12/01/2022,437169,Sales,2,470.00,Cash,10:10:08,pm,14,DIRECTOR,1405,COCA COLA PET 60CL,1,170.00
2,12/01/2022,437169,Sales,2,470.00,Cash,10:10:08,pm,14,DIRECTOR,90,MCVITE'S DIGESTIVE 104G,1,300.00
3,12/01/2022,437168,Sales,1,100.00,Cash,10:09:10,pm,14,DIRECTOR,7347,ROENEX SMALLEST PEANUTS,1,100.00
4,12/01/2022,437173,Sales,2,"2,100.00",Debit Card,Card,10:14:36,pm,14,DIRECTOR,2855,EASY ON SPRAY STARCH NEW,BLUE1


In [32]:
main_df.shape

(426032, 14)

In [ ]:
#save impported dataframe to CSV
imported_data_path = Path.joinpath(Path.cwd().parent,"data/processed/imported_oleh_sales.csv")
main_df.to_csv(imported_data_path)

### Structural and Quality investigation

In [ ]:
#import the save CSV
imported_data_path = Path.joinpath(Path.cwd().parent,"data/processed/imported_oleh_sales.csv")
import_df = pd.read_csv(imported_data_path)

/var/folders/0c/vr9_9rds4t10m_q7fxzlb3x00000gn/T/ipykernel_4194/3325350122.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  import_df = pd.read_csv(imported_data_path)


In [ ]:

import_df.shape

(426032, 15)